# Calculating population density

We want to know how many individuals are exposed to flares at each of the study regions.

To do this we want to:
- read in all population data from the study regions
- calculate the total population density within each of the study regions
-  using literatiure select an appropriate threshhold for AOD to use as a cutoff point for the "high levels of pollution"
- from this calculate total no. of people and % of people in area affected by high levels of pollution

In [24]:
import os

# Reading in all population data
import geopandas as gpd
base_dir = os.path.dirname(os.getcwd())

In [54]:
gwer_road = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_gwer_polygon.geojson")
kalak = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_kalak_polygon.geojson")
lalish = gpd.read_file(f"{base_dir}/pop_dens_data/polygon_geojson/POP_DEN_lalish_polygon.geojson")

In [55]:
gwer_road.head()

,VALUE,geometry
0,441.916229,"MULTIPOLYGON (((43.78208 36.24139, 43.78208 36..."
1,419.327271,"MULTIPOLYGON (((43.78547 36.24193, 43.78977 36..."
2,511.886505,"MULTIPOLYGON (((43.74875 36.23515, 43.74875 36..."
3,441.208771,"MULTIPOLYGON (((43.75148 36.23655, 43.75708 36..."
4,245.390091,"MULTIPOLYGON (((43.76542 36.23875, 43.76542 36..."


In [56]:
sum(gwer_road["VALUE"])

121183.42779943

In [57]:
sum(gwer_road["VALUE"])

121183.42779943

In [58]:
sum(kalak["VALUE"])

219058.2630378402

In [59]:
sum(lalish["VALUE"])

40925.76306725002

Next we want to read in the `AOD` data which quanitifies atmospheric haziness and gives us an idea of the level of pollution in the atmosphere.


In [60]:
gwer_road_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_gwer_polygon.geojson")
kalak_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_kalak_polygon.geojson")
lalish_aod = gpd.read_file(f"{base_dir}/aod_data/aod_polygons/AOD_lalish_polygon.geojson")

In order to understand the exposure to `AOD` we need to count the population exposed to `AOD` concentrations above an average of the following threshhold:
- PM2.5 (μg m−3) 30.5 = AOD (5 km) 172.1

In [61]:
gwer_road_aod

,VALUE,index_right,longitude,latitude,month_year,Optical_Depth_047,Optical_Depth_055,Date,geometry
0,441.916229,29694,43.781776,36.241317,2019-07,288.536585,213.317073,2019-07-01T00:00:00,"MULTIPOLYGON (((43.78208 36.24139, 43.78208 36..."
1,441.916229,29696,43.781776,36.241317,2019-09,202.415385,149.692308,2019-09-01T00:00:00,"MULTIPOLYGON (((43.78208 36.24139, 43.78208 36..."
2,441.916229,29697,43.781776,36.241317,2019-10,164.925000,121.975000,2019-10-01T00:00:00,"MULTIPOLYGON (((43.78208 36.24139, 43.78208 36..."
3,441.916229,29698,43.781776,36.241317,2019-11,114.725490,84.823529,2019-11-01T00:00:00,"MULTIPOLYGON (((43.78208 36.24139, 43.78208 36..."
4,441.916229,29709,43.781776,36.241317,2020-10,175.074074,129.462963,2020-10-01T00:00:00,"MULTIPOLYGON (((43.78208 36.24139, 43.78208 36..."
...,...,...,...,...,...,...,...,...,...
22809,211.958817,31625,43.799922,36.024374,2021-10,144.604651,106.883721,2021-10-01T00:00:00,"MULTIPOLYGON (((43.79875 36.02403, 43.79875 36..."
22810,211.958817,31626,43.799922,36.024374,2021-11,124.611111,92.111111,2021-11-01T00:00:00,"MULTIPOLYGON (((43.79875 36.02403, 43.79875 36..."
22811,211.958817,31627,43.799922,36.024374,2021-12,120.307692,88.923077,2021-12-01T00:00:00,"MULTIPOLYGON (((43.79875 36.02403, 43.79875 36..."
22812,211.958817,31628,43.799922,36.024374,2022-01,180.607143,133.464286,2022-01-01T00:00:00,"MULTIPOLYGON (((43.79875 36.02403, 43.79875 36..."


In [71]:
gwer_road_mean_aod = gwer_road_aod.groupby(["longitude", "latitude", "VALUE"])["Optical_Depth_047"].mean().reset_index()

kalak_mean_aod = kalak_aod.groupby(["longitude", "latitude", "VALUE"])["Optical_Depth_047"].mean().reset_index()

lalish_mean_aod = lalish_aod.groupby(["longitude", "latitude", "VALUE"])["Optical_Depth_047"].mean().reset_index()

Calculate the percentage of population exposed to rates of PM2.5 (using the AOD-> PM2.5 conversion) exceeding the WHO annual mean concentrations recommendations.

In [65]:
gwer_road_harmful_aod = gwer_road_mean_aod[gwer_road_mean_aod["Optical_Depth_047"]>175]
kalak_harmful_aod = kalak_mean_aod[kalak_mean_aod["Optical_Depth_047"]>175]
lalish_harmful_aod = lalish_mean_aod[lalish_mean_aod ["Optical_Depth_047"]>175]

In [82]:
sum(gwer_road_harmful_aod["VALUE"])/sum(gwer_road_mean_aod['VALUE'])

1.0

In [83]:
sum(kalak_harmful_aod["VALUE"])/sum(kalak_mean_aod['VALUE'])

1.0

In [81]:
sum(lalish_harmful_aod["VALUE"])/sum(lalish_mean_aod['VALUE'])

0.9951336173308959